In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

params = {"ot": "OTTESTING01",
            "datadate": "2023-05-11"}
item_activity_file = "data/input/Matriz de Conversion Clarificador Rev.B1.xlsx"

In [2]:
# Importar la clase desde el archivo add_columns_to_dataframe.py
from utils.add_columns_to_dataframe import AddColumnsToDataFrame

from excel_processor.excel_processor import ExcelProcessor
from excel_processor.schemas import itemizado_schema, activity_schema

In [3]:
# Procesa el archivo Excel
processor = ExcelProcessor(item_activity_file, itemizado_schema)
result = processor.process()

if result.error_message:
    print(f"Error: {result.error_message}")
else:
    item_summary = result.dataframe

In [4]:
# Procesa el archivo Excel
processor = ExcelProcessor(item_activity_file, activity_schema)
result = processor.process()

if result.error_message:
    print(f"Error: {result.error_message}")
else:
    activity_summary = result.dataframe

Error: La columna qty_activity contiene valores 0 en las celdas: qty_activity3, qty_activity5, qty_activity8, qty_activity10, qty_activity12, qty_activity14, qty_activity16, qty_activity18, qty_activity19, qty_activity20, qty_activity22, qty_activity23, qty_activity26, qty_activity28, qty_activity30, qty_activity32, qty_activity36, qty_activity39, qty_activity42, qty_activity44, qty_activity51, qty_activity53, qty_activity55, qty_activity56, qty_activity58, qty_activity61, qty_activity63, qty_activity68, qty_activity70, qty_activity72, qty_activity74, qty_activity76, qty_activity80, qty_activity85, qty_activity87, qty_activity89, qty_activity91, qty_activity93, qty_activity95, qty_activity98, qty_activity100, qty_activity102, qty_activity104, qty_activity107, qty_activity109, qty_activity111, qty_activity113, qty_activity116, qty_activity118, qty_activity120, qty_activity122, qty_activity125, qty_activity127, qty_activity129, qty_activity131, qty_activity134, qty_activity136, qty_activ

In [5]:
# Crear instancia de la clase e ingresar el dataframe y el diccionario de parámetros
add_columns_to_df = AddColumnsToDataFrame(dataframe=item_summary, parameters=params) # type: ignore

# Agregar columnas al dataframe
item_summary_with_columns = add_columns_to_df.add_columns()